## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import h5py

## Exploring and Extracting Behavioral Data from Electrophysiology Paper

In [2]:
ephys_file = h5py.File("/Users/Devin Chang/Downloads/Chewie_CO_20150630.mat", 'r')
print(ephys_file.keys())

<KeysViewHDF5 ['#refs#', 'trial_data']>


In [3]:
print(ephys_file['trial_data'].keys())

<KeysViewHDF5 ['M1_lfp', 'M1_lfp_guide', 'M1_spikes', 'M1_unit_guide', 'bin_size', 'date_time', 'idx_endTime', 'idx_goCueTime', 'idx_movement_on', 'idx_startTime', 'idx_tgtOnTime', 'idx_trial_end', 'idx_trial_start', 'monkey', 'result', 'tgtDir', 'vel']>


In [4]:
behavioral_data_list = ['idx_endTime', 'idx_goCueTime', 'idx_movement_on', 'idx_startTime', 'idx_tgtOnTime', 'idx_trial_end', 'idx_trial_start']
behavioral_data_df = np.array([])
for info in behavioral_data_list:
    data = ephys_file['trial_data'][info]
    extracted_data = []
    for i in range(data.shape[0]):
        row_data = []
        for j in range(data.shape[1]):
            ref = data[i, j]  # Get the reference
            if isinstance(ref, h5py.Reference):  # Check if it's a reference
                actual_data = ephys_file[ref][()]  # Dereference and get data
                row_data.append(actual_data)
            else:
                row_data.append(ref)  # Directly append non-referenced data
        extracted_data.append(row_data)

    # Convert to a NumPy array and then to DataFrame (if applicable)
    np_data = np.array(extracted_data).reshape(178,)#, dtype=object)
    behavioral_data_df = np.hstack((behavioral_data_df, np_data))
behavioral_data_df = pd.DataFrame(behavioral_data_df.reshape(len(behavioral_data_list), 178).T, 
                                  columns=behavioral_data_list)

In [5]:
behavioral_data_df

,idx_endTime,idx_goCueTime,idx_movement_on,idx_startTime,idx_tgtOnTime,idx_trial_end,idx_trial_start
0,142.0,107.0,108.0,1.0,73.0,NaN,1.0
1,167.0,135.0,141.0,1.0,93.0,176.0,1.0
2,114.0,75.0,88.0,1.0,53.0,122.0,1.0
3,134.0,97.0,106.0,1.0,64.0,142.0,1.0
4,118.0,85.0,93.0,1.0,45.0,126.0,1.0
...,...,...,...,...,...,...,...
173,114.0,75.0,87.0,1.0,48.0,122.0,1.0
174,136.0,98.0,112.0,1.0,70.0,NaN,1.0
175,127.0,91.0,101.0,1.0,54.0,135.0,1.0
176,136.0,98.0,113.0,1.0,48.0,144.0,1.0


In [6]:
trial_starttime = np.array(behavioral_data_df['idx_startTime']) - 1
trial_endtime = np.array(behavioral_data_df['idx_endTime']) - 1
trial_movement_on = np.array(behavioral_data_df['idx_movement_on'])

## Preprocessing and Normalization

In [50]:
with h5py.File("/Users/Devin Chang/Downloads/Chewie_CO_20150630.mat", 'r') as mat:
    data_list = []
    data = mat['trial_data']['M1_lfp']
    # Convert to NumPy array (MATLAB stores in column-major order, so transpose if needed)
    if isinstance(data[0, 0], h5py.Reference):
        # Extract actual data from references
        extracted_data = []
        for i in range(data.shape[0]):  # Iterate over rows
            row_data = []
            for j in range(data.shape[1]):  # Iterate over columns
                ref = data[i, j]  # Get the reference
                if isinstance(ref, h5py.Reference):  # Check if it's a reference
                    actual_data = mat[ref][()]  # Dereference and get data
                    row_data.append(actual_data)
                else:
                    row_data.append(ref)  # Directly append non-referenced data
            extracted_data.append(row_data)

        # Convert to a NumPy array and then to DataFrame (if applicable)
        np_data = extracted_data#, dtype=object)
        lfp_df = pd.DataFrame(np_data)

In [40]:
pd.DataFrame(lfp_df[0][0]).T

,0,1,2,3,4,5,6,7,8,9,...,854,855,856,857,858,859,860,861,862,863
0,8.257605,42.878782,63.301639,1.283340,15.801977,31.863385,44.897776,72.045651,51.539563,15.346799,...,701.082675,14.753389,78.166445,61.154394,40.086982,54.929478,31.155542,31.057044,142.168923,27.141508
1,1.281180,42.859230,56.917981,2.666293,15.971701,46.606807,31.429457,67.583616,41.308509,6.611114,...,692.364498,14.902779,80.244602,58.463964,41.486497,76.598303,25.550656,31.547641,155.466928,27.080599
2,-7.685382,42.711337,55.778981,6.675004,16.990480,43.712009,46.306036,55.344703,38.607804,-1.384134,...,706.097289,13.522223,81.252464,50.661874,54.857652,104.199138,22.785694,43.015944,139.954524,21.760537
3,-10.719503,42.380179,54.586844,13.088107,19.931322,50.536605,68.939265,51.641724,35.171302,-3.401443,...,694.389221,17.627799,81.427882,49.364873,59.137833,126.691178,14.637440,50.244159,112.173875,21.220982
4,-5.274688,41.879501,44.956520,21.140904,20.355948,76.888430,74.356965,63.603587,35.040695,6.266364,...,704.748608,9.118808,81.356481,45.671050,68.260170,117.099163,12.486702,63.833741,92.023595,22.276900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,-15.950196,72.235273,27.111106,9.582767,22.616733,25.843046,36.013704,68.937066,24.007891,-30.653910,...,697.568048,13.501116,95.454146,34.019149,27.649677,183.150706,87.158863,71.174319,152.535457,38.811866
145,-16.506689,73.371396,32.477607,5.576015,25.991430,31.107557,45.646061,76.902404,33.498279,-25.344021,...,776.513179,23.958855,98.599920,48.387162,30.149217,163.138337,57.857841,76.932580,155.411646,29.714033
146,-9.366235,74.333296,34.125276,3.938636,24.622420,36.478300,47.978507,71.507306,32.556722,-13.786378,...,780.807683,28.026654,100.082653,54.397375,37.722513,118.382978,47.425274,67.600526,155.398468,23.152977
147,0.663022,74.671967,39.916477,2.346640,22.801099,49.223705,44.739138,57.394521,31.533487,-1.961522,...,756.102813,26.088976,99.513324,55.436849,36.898057,86.671525,43.442372,65.310640,156.483704,19.386516


In [28]:
num_trials = 10
num_neurons = 864

all_trials_lfp_list = []
y_labels_list = []
for i in range(num_trials):
    X = pd.DataFrame(lfp_df[0][i]).T
    X = X.iloc[int(trial_starttime[i]):int(trial_endtime[i])]
    y = np.zeros(X.shape[0])
    y[int(trial_movement_on[i]):] = 1
    X = X.reset_index().drop(columns='index')
    y_labels_list.append(y)
    
    df_list = []
    for time in X.index:
        current_df = pd.DataFrame({'time':f"{i}_{time}", 'neuron':X.columns,
                                   'value': X.iloc[time, :]})
        df_list.append(current_df)
    long_df = pd.concat(df_list, ignore_index=True)
    all_trials_lfp_list.append(long_df)
all_trials_lfp_df = pd.concat(all_trials_lfp_list, ignore_index=True)
y_labels = np.concatenate(y_labels_list)

In [19]:
all_trials_lfp_df

,time,neuron,value
0,0_0,0,8.257605
1,0_0,1,42.878782
2,0_0,2,63.301639
3,0_0,3,1.283340
4,0_0,4,15.801977
...,...,...,...
1156027,9_128,859,101.711083
1156028,9_128,860,54.748275
1156029,9_128,861,112.671200
1156030,9_128,862,182.233188


## Feature Extraction

In [41]:
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [12]:
extracted_features = extract_features(all_trials_lfp_df, column_id="time", column_sort="neuron")

Feature Extraction: 100%|██████████| 20/20 [10:01<00:00, 30.08s/it]


In [30]:
# Drop NaN and impute missing values
extracted_features = impute(extracted_features)
selected_features = select_features(extracted_features, y_labels)

In [42]:
selected_features

,"value__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""mean""","value__fft_coefficient__attr_""imag""__coeff_1","value__fft_coefficient__attr_""angle""__coeff_1","value__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""mean""","value__linear_trend__attr_""pvalue""","value__linear_trend__attr_""rvalue""",value__index_mass_quantile__q_0.4,"value__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""mean""","value__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""mean""","value__linear_trend__attr_""slope""",...,"value__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0","value__cwt_coefficients__coeff_12__w_2__widths_(2, 5, 10, 20)","value__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2","value__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""",value__index_mass_quantile__q_0.2,"value__linear_trend__attr_""stderr""","value__fft_coefficient__attr_""abs""__coeff_31","value__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2","value__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4","value__fft_coefficient__attr_""angle""__coeff_36"
0_0,0.214164,604.877189,0.566308,0.171785,0.008815,0.089057,0.328704,2.407552,1.255391,0.253191,...,111.139086,33.361142,24.354178,103.614263,0.228009,0.096449,24884.891704,951.803627,24.863067,-23.651618
0_1,0.207495,-392.612666,-0.375504,0.166516,0.012323,0.085114,0.314815,2.285067,1.191871,0.240395,...,113.218383,34.119499,25.335913,104.765015,0.228009,0.095850,25185.816182,1035.782175,27.067171,-22.596295
0_10,0.222966,2866.109745,2.557508,0.179775,0.007485,0.090931,0.405093,2.623925,1.369527,0.276318,...,111.375429,62.147554,27.345182,104.894258,0.228009,0.103072,26229.038671,1199.419457,29.839969,-22.899288
0_100,0.219605,271.891621,0.243311,0.177583,0.008103,0.090026,0.329861,2.494316,1.293774,0.260629,...,120.507179,-29.866637,27.104818,106.678024,0.228009,0.098205,25226.196660,1141.130388,28.224518,-26.381688
0_101,0.222853,819.250892,0.724140,0.180419,0.006958,0.091756,0.335648,2.555614,1.321944,0.266154,...,123.129309,-22.570520,25.341257,106.449306,0.228009,0.098381,25291.860547,990.421457,26.472055,-26.641772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9_95,0.183206,-5240.324988,-4.815785,0.150586,0.025354,0.076069,0.311343,2.144896,1.113966,0.224564,...,153.639132,-25.217948,26.461215,126.615647,0.228009,0.100258,26507.807835,1204.669098,28.214647,-21.695325
9_96,0.189550,-3871.851784,-3.549908,0.155477,0.020534,0.078799,0.312500,2.216588,1.153327,0.232557,...,181.280322,-27.298082,30.000996,123.897751,0.228009,0.100208,25975.551415,1513.905339,30.230910,-23.289839
9_97,0.188597,-3469.499260,-3.270927,0.152254,0.023863,0.076861,0.312500,2.175690,1.136959,0.229443,...,190.622158,-21.533076,28.667721,120.541362,0.228009,0.101374,25240.662000,1332.795487,28.614276,-25.391476
9_98,0.193907,-2324.337742,-2.236886,0.154764,0.021100,0.078451,0.328704,2.216312,1.158421,0.233798,...,198.848228,-14.800561,27.668182,115.791482,0.228009,0.101193,24487.549090,1196.410364,29.082270,-26.810713


## Classification with Random Forest Classifier

In [43]:
print(y_labels)

[0. 0. 0. ... 1. 1. 1.]


In [38]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, y_labels, test_size=0.2, random_state=0, stratify=y_labels)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9291044776119403
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96       215
         1.0       0.97      0.66      0.79        53

    accuracy                           0.93       268
   macro avg       0.95      0.83      0.87       268
weighted avg       0.93      0.93      0.92       268



# Calcium Imaging Dataset

## Loading Datasets

In [47]:
calcium_imaging_file = h5py.File('/Users/Devin Chang/Documents/Marmoset1/Marmoset1-data.mat', 'r')
print(calcium_imaging_file.keys())

<KeysViewHDF5 ['#refs#', 'roi', 'session_data']>


In [141]:
def make_idx_list(ser):
    return list(ser.dropna().apply(int))

In [143]:
roi_idx_df = pd.read_csv("/Users/Devin Chang/Documents/Marmoset1/roi_idx.csv").set_index('label')
roi_idx_df['idx'] = roi_idx_df.apply(make_idx_list, axis=1)
roi_idx_df = roi_idx_df[['idx']]

In [144]:
roi_idx_df

,idx
label,
8aV,"[82, 83, 84, 85, 86, 87, 88, 89, 90, 144, 145,..."
8b,"[108, 109, 171, 172, 173, 174, 175, 176, 234, ..."
8c,"[153, 154, 217, 218, 280, 281, 282, 343, 344, ..."
6Dr,"[38, 39, 91, 92, 93, 94, 95, 96, 97, 98, 99, 1..."
6Dc,"[602, 603, 604, 605, 664, 665, 666, 667, 668, ..."
6V,"[272, 273, 274, 275, 276, 277, 278, 336, 337, ..."
6M,"[496, 554, 555, 556, 557, 558, 559, 560, 618, ..."
4ab,"[1239, 1240, 1241, 1302, 1303, 1304, 1305, 130..."
4c,"[848, 912, 913, 914, 976, 977, 978, 979, 1040,..."


In [158]:
num_df = 10
calcium_imaging_df_list = []
calcium_imaging_folder = '/Users/Devin Chang/Documents/Marmoset1/calcium_imaging_data'
for i in range(1, num_df + 1):
    df = pd.read_csv(calcium_imaging_folder + f'/trial_{i}.csv', header=None)
    calcium_imaging_df_list.append(df)

In [159]:
calcium_imaging_df_list[0]

,0,1,2,3,4,5,6,7,8,9,...,286,287,288,289,290,291,292,293,294,295
0,2.919705e-06,0.000002,0.000002,1.882468e-06,1.534436e-06,1.866604e-06,1.836362e-06,1.518441e-06,0.000002,1.427832e-06,...,1.743409e-07,5.305133e-07,2.884557e-07,3.565747e-07,1.984114e-07,3.736354e-07,5.815407e-07,2.904824e-07,4.237652e-07,4.411141e-07
1,-3.533586e-05,-0.000038,-0.000040,-3.371209e-05,-3.306586e-05,-3.254881e-05,-3.372005e-05,-3.407252e-05,-0.000032,-3.106179e-05,...,-3.749966e-06,-3.166592e-06,-6.123564e-06,-3.330290e-06,-1.484726e-06,6.837561e-07,4.108693e-06,8.999734e-06,6.593380e-06,6.518577e-06
2,-1.344078e-05,-0.000006,-0.000015,-9.995447e-06,-1.132800e-05,-1.189011e-05,-7.270114e-06,-9.712302e-06,-0.000007,-7.624892e-06,...,-1.676613e-06,-4.540027e-06,-1.374185e-06,3.587920e-07,-5.663967e-07,-5.561172e-06,-3.837139e-06,6.951782e-07,-1.204158e-06,-6.490919e-06
3,-3.441154e-06,-0.000009,-0.000004,-1.825091e-06,-4.640276e-06,-1.279701e-06,-9.647399e-07,-5.995730e-07,-0.000002,-4.587551e-06,...,-4.863314e-07,-3.221172e-06,-1.246747e-06,-5.091118e-07,-3.339409e-06,-1.216375e-06,-2.089731e-06,-2.778248e-06,-4.417034e-06,-4.803241e-06
4,1.034998e-07,0.000005,0.000002,5.469345e-07,4.603168e-07,-5.653084e-07,-8.153557e-08,-1.418958e-06,-0.000001,-1.471316e-07,...,-1.401968e-06,2.312425e-07,-1.302753e-06,-2.918739e-07,1.265286e-06,1.780763e-07,3.285432e-08,1.779186e-07,1.256659e-06,1.025105e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4092,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4093,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4094,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [169]:
ci_behavior_df = pd.read_excel('/Users/Devin Chang/Documents/Marmoset1/calcium_imaging_behavioral_trials.xlsx')
ci_behavior_df

,Trial,onset_cue,onset_pull,onset_push,success,trial_type,first_choice
0,1,100,-1,183,1,1,1
1,2,99,-1,152,1,1,1
2,3,101,-1,388,1,1,1
3,4,97,-1,168,1,1,1
4,5,96,-1,165,1,1,1
5,6,99,-1,136,1,1,1
6,7,102,-1,139,1,1,1
7,8,150,-1,198,1,1,1
8,9,100,-1,138,1,1,1
9,10,99,-1,142,1,1,1


## Data Preprocessing

In [172]:
M1_roi_idx = np.array(roi_idx_df.loc['4ab']['idx']) - 1
PMdc_roi_idx = np.array(roi_idx_df.loc['6Dc']['idx']) - 1
PMdr_roi_idx = np.array(roi_idx_df.loc['6Dr']['idx']) - 1

In [171]:
print(M1_roi_idx)

[1238 1239 1240 1301 1302 1303 1304 1305 1306 1307 1308 1309 1310 1311
 1312 1313 1314 1365 1366 1367 1368 1369 1370 1371 1372 1373 1374 1375
 1376 1377 1378 1379 1380 1381 1382 1383 1384 1385 1386 1387 1388 1389
 1390 1429 1430 1431 1432 1433 1434 1435 1436 1437 1438 1439 1440 1441
 1442 1443 1444 1445 1446 1447 1448 1449 1450 1451 1452 1453 1454 1455
 1493 1494 1495 1496 1497 1498 1499 1500 1501 1502 1503 1504 1505 1506
 1507 1508 1509 1510 1511 1512 1513 1514 1515 1516 1517 1518 1519 1557
 1558 1559 1560 1561 1562 1563 1564 1565 1566 1567 1568 1569 1570 1571
 1572 1573 1574 1575 1576 1577 1578 1579 1580 1581 1582 1583 1621 1622
 1623 1624 1625 1626 1627 1628 1629 1630 1631 1632 1633 1634 1635 1636
 1637 1638 1639 1640 1641 1642 1643 1644 1645 1646 1647 1685 1686 1687
 1688 1689 1690 1691 1692 1693 1694 1695 1696 1697 1698 1699 1700 1701
 1702 1703 1704 1705 1706 1707 1708 1709 1710 1711 1750 1751 1752 1753
 1754 1755 1756 1757 1758 1759 1760 1761 1762 1763 1764 1765 1766 1767
 1768 

In [178]:
ci_M1_df_list = [] 
for data in calcium_imaging_df_list:
    data = data.loc[M1_roi_idx].T
    ci_M1_df_list.append(data)
ci_M1_df_list[0]

,1238,1239,1240,1301,1302,1303,1304,1305,1306,1307,...,2286,2342,2343,2344,2345,2346,2347,2348,2349,2350
0,-0.397130,-0.653119,-0.829642,-0.237469,-0.545765,-0.593284,-0.585078,-0.701838,-0.723307,-0.659907,...,-0.660166,-0.628794,-0.584169,-0.505326,-0.792260,-0.951849,-1.263783,-1.079828,-0.911253,-0.888962
1,-0.473834,-0.740661,-0.927336,-0.312852,-0.637405,-0.692798,-0.694614,-0.790996,-0.808154,-0.773173,...,-0.425994,-0.798146,-0.718317,-0.625794,-0.983194,-1.172341,-1.427082,-1.286168,-1.176788,-1.181069
2,-0.660657,-0.900975,-1.096124,-0.473538,-0.811514,-0.874250,-0.847355,-0.955417,-1.011256,-0.993379,...,-0.907777,-0.994433,-0.897538,-0.786983,-1.136560,-1.334228,-1.640163,-1.517312,-1.331130,-0.883874
3,-0.665365,-0.870937,-1.025627,-0.476715,-0.802048,-0.862595,-0.814823,-0.890919,-0.949035,-0.953278,...,-1.432244,-0.903004,-0.815966,-0.728043,-1.075401,-1.324848,-1.569223,-1.423839,-1.279348,-1.273397
4,-0.386687,-0.610903,-0.723226,-0.196402,-0.442993,-0.547433,-0.551416,-0.599971,-0.655308,-0.647794,...,-0.359641,-0.616732,-0.532697,-0.488219,-0.893803,-1.112789,-1.317984,-1.153778,-0.928276,-0.625688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,0.168888,0.356962,0.298350,-0.039746,0.310585,0.373754,0.354586,0.315539,0.266251,0.253512,...,-0.305301,-0.506602,-0.286834,-0.174327,-0.324650,-0.259279,-0.367385,-0.384704,-0.384616,-0.092873
292,0.333215,0.495604,0.428269,0.203875,0.495434,0.526268,0.503897,0.435872,0.368246,0.374198,...,0.115647,-0.397771,-0.217740,-0.093731,-0.147775,-0.079861,-0.264057,-0.255179,-0.173734,0.194228
293,0.683110,0.829113,0.734896,0.601391,0.834797,0.811999,0.744044,0.679031,0.609262,0.597736,...,0.216134,-0.242291,-0.074396,0.102053,0.142143,0.186825,0.036203,0.014927,0.023112,0.132133
294,0.645374,0.777535,0.700724,0.694264,0.852684,0.769015,0.736064,0.677237,0.602074,0.594615,...,0.018170,-0.212634,-0.096152,0.065065,0.141888,0.153020,0.011182,0.034871,-0.043173,-0.076007


In [183]:
trial_push_onset = list(ci_behavior_df['onset_push'])

y_list = []
for i in range(len(trial_push_onset)):
    temp_df = ci_M1_df_list[i]
    y = np.zeros(temp_df.shape[0])
    y[trial_push_onset[i]:] = 1
    y_list.append(y)